In [1]:
FILE_PATH = ''
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        FILE_PATH = os.path.join(dirname, filename)

In [2]:
pip install tensorflow==2.16.1

Note: you may need to restart the kernel to use updated packages.


In [4]:
import tensorflow as tf

In [5]:
tf.__version__

'2.16.1'

In [6]:
# Method 2: Using open and close explicitly
file = open(FILE_PATH, 'r')
content = file.read()
file.close()

In [7]:
content[:660].split('\n')

['The Happy Prince.',
 'HIGH above the city, on a tall column, stood the statue of the Happy Prince.  He was gilded all over with thin leaves of fine gold, for eyes he had two bright sapphires, and a large red ruby glowed on his sword-hilt.',
 'He was very much admired indeed.  “He is as beautiful as a weathercock,” remarked one of the Town Councillors who wished to gain a reputation for having artistic tastes; “only not quite so useful,” he added, fearing lest people should think him unpractical, which he really was not.',
 '“Why can’t you be like the Happy Prince?” asked a sensible mother of her little boy who was crying for the moon.  “The Happy Prince never drea']

In [8]:
#Remove punctuations
import re
_content = re.sub(r'[^\w\s]', '', content)
content_clean = _content.lower()
content_clean = content_clean[:90000]

In [9]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer

In [10]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts([content_clean])
unique_cnt = len(tokenizer.word_index)
print("Total unique words in the document is ", unique_cnt)

Total unique words in the document is  2372


In [11]:
total_sentences = content_clean.split('\n')
print("Total sentences are ",len(total_sentences))

Total sentences are  489


In [12]:
input_sequences = []
for sentence in total_sentences:
    tokenized_sentence = tokenizer.texts_to_sequences([sentence])[0]
    for i in range(1,len(tokenized_sentence)):
        input_sequences.append(tokenized_sentence[:i+1])

In [13]:
max_len = max([len(x) for x in input_sequences])
print("Size of the longest sentence is ",max_len)

Size of the longest sentence is  169


In [14]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
padded_input_sequences = pad_sequences(input_sequences, maxlen = max_len, padding='pre')

In [15]:
padded_input_sequences

array([[   0,    0,    0, ...,    0,    1,   81],
       [   0,    0,    0, ...,    1,   81,   50],
       [   0,    0,    0, ...,    0,  285,  618],
       ...,
       [   0,    0,    0, ...,   82, 1172,  617],
       [   0,    0,    0, ..., 1172,  617,   34],
       [   0,    0,    0, ...,  617,   34,   27]], dtype=int32)

In [16]:
X = padded_input_sequences[:,:-1]

In [17]:
X.shape

(17195, 168)

In [18]:
y = padded_input_sequences[:,-1]

In [19]:
y.shape

(17195,)

In [20]:
from tensorflow.keras.utils import to_categorical
y = to_categorical(y,num_classes=unique_cnt+1)

In [21]:
y.shape

(17195, 2373)

In [35]:
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout
from keras.regularizers import l2

In [23]:
X.shape

(17195, 168)

In [24]:
y.shape

(17195, 2373)

In [27]:
model = Sequential()
model.add(Embedding(unique_cnt+1, 128, input_shape=(max_len,)))
model.add(LSTM(150))
model.add(Dense(unique_cnt+1, activation='softmax'))

In [28]:
model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])

In [29]:
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_2 (Embedding)         │ (None, 169, 128)       │       303,744 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_2 (LSTM)                   │ (None, 150)            │       167,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 2373)           │       358,323 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 829,467 (3.16 MB)

 Trainable params: 829,467 (3.16 MB)

 Non-trainable params: 0 (0.00 B)

In [30]:
model.fit(X,y,epochs=100)

Epoch 1/100
538/538 ━━━━━━━━━━━━━━━━━━━━ 8s 10ms/step - accuracy: 0.0726 - loss: 6.4331
Epoch 2/100
538/538 ━━━━━━━━━━━━━━━━━━━━ 5s 9ms/step - accuracy: 0.0892 - loss: 5.6796
Epoch 3/100
538/538 ━━━━━━━━━━━━━━━━━━━━ 5s 9ms/step - accuracy: 0.1240 - loss: 5.2705
Epoch 4/100
538/538 ━━━━━━━━━━━━━━━━━━━━ 5s 9ms/step - accuracy: 0.1471 - loss: 4.8845
Epoch 5/100
538/538 ━━━━━━━━━━━━━━━━━━━━ 5s 9ms/step - accuracy: 0.1612 - loss: 4.6217
Epoch 6/100
538/538 ━━━━━━━━━━━━━━━━━━━━ 5s 9ms/step - accuracy: 0.1868 - loss: 4.2840
Epoch 7/100
538/538 ━━━━━━━━━━━━━━━━━━━━ 5s 9ms/step - accuracy: 0.2032 - loss: 4.0827
Epoch 8/100
538/538 ━━━━━━━━━━━━━━━━━━━━ 5s 9ms/step - accuracy: 0.2221 - loss: 3.8375
Epoch 9/100
538/538 ━━━━━━━━━━━━━━━━━━━━ 5s 9ms/step - accuracy: 0.2453 - loss: 3.6589
Epoch 10/100
538/538 ━━━━━━━━━━━━━━━━━━━━ 5s 9ms/step - accuracy: 0.2663 - loss: 3.4123
Epoch 11/100
538/538 ━━━━━━━━━━━━━━━━━━━━ 5s 9ms/step - accuracy: 0.2958 - loss: 3.2344
Epoch 12/100
538/538 ━━━━━━━━━━━━━━━━━━━

In [49]:
model_two = Sequential()
model_two.add(Embedding(unique_cnt+1, 128, input_shape=(max_len,)))
model_two.add(LSTM(256))
model_two.add(Dense(unique_cnt+1, activation='softmax'))

In [50]:
model_two.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])

In [51]:
model_two.summary()

Model: "sequential_11"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_11 (Embedding)        │ (None, 169, 128)       │       303,744 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_11 (LSTM)                  │ (None, 256)            │       394,240 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 2373)           │       609,861 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,307,845 (4.99 MB)

 Trainable params: 1,307,845 (4.99 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model_two.fit(X,y,epochs=42)

Epoch 1/42
538/538 ━━━━━━━━━━━━━━━━━━━━ 8s 12ms/step - accuracy: 0.0730 - loss: 6.3935
Epoch 2/42
538/538 ━━━━━━━━━━━━━━━━━━━━ 6s 12ms/step - accuracy: 0.0973 - loss: 5.5485
Epoch 3/42
538/538 ━━━━━━━━━━━━━━━━━━━━ 6s 12ms/step - accuracy: 0.1389 - loss: 5.0685
Epoch 4/42
538/538 ━━━━━━━━━━━━━━━━━━━━ 6s 12ms/step - accuracy: 0.1614 - loss: 4.6158
Epoch 5/42
538/538 ━━━━━━━━━━━━━━━━━━━━ 7s 12ms/step - accuracy: 0.1853 - loss: 4.2197
Epoch 6/42
538/538 ━━━━━━━━━━━━━━━━━━━━ 7s 12ms/step - accuracy: 0.2110 - loss: 3.8293
Epoch 7/42
538/538 ━━━━━━━━━━━━━━━━━━━━ 6s 12ms/step - accuracy: 0.2569 - loss: 3.4434
Epoch 8/42
538/538 ━━━━━━━━━━━━━━━━━━━━ 6s 12ms/step - accuracy: 0.3177 - loss: 3.0544
Epoch 9/42
396/538 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.3969 - loss: 2.6954

In [34]:
import time
import numpy as np
text = "what is the fee"

for i in range(10):
  # tokenize
    token_text = tokenizer.texts_to_sequences([text])[0]
  # padding
    padded_token_text = pad_sequences([token_text], maxlen=max_len, padding='pre')
  # predict
    pos = np.argmax(model.predict(padded_token_text))
    for word,index in tokenizer.word_index.items():
        if index == pos:
            text = text + " " + word
            print(text)
            time.sleep(2)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 121ms/step
what is the fee use
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
what is the fee use of
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
what is the fee use of a
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
what is the fee use of a statue
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
what is the fee use of a statue if
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
what is the fee use of a statue if it
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
what is the fee use of a statue if it cannot
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
what is the fee use of a statue if it cannot keep
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
what is the fee use of a statue if it cannot keep the
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
what is the fee use of a statue if it cannot keep the rain
